### Inline CSV Data — sales_transactions.csv

In [0]:
# Save inline CSV to Databricks FileStore
csv_data = """transaction_id,customer_name,region,product,category,quantity,unit_price,date
1,Rajesh,North,Laptop,Electronics,1,55000,2024-01-12
2,Sneha,West,Refrigerator,Electronics,1,32000,2024-02-05
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17
4,Divya,North,Mobile,Electronics,2,20000,2024-03-22
5,Vikram,East,Washing Machine,Electronics,1,28000,2024-02-28
6,Preeti,West,Sneakers,Fashion,2,4000,2024-01-31
7,Aman,South,TV,Electronics,1,45000,2024-02-15
8,Isha,North,Notebook,Stationery,10,60,2024-01-10
9,Kunal,East,Pencil,Stationery,20,10,2024-03-05
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19
"""

dbutils.fs.put("dbfs:/FileStore/tables/sales_transactions.csv", csv_data, overwrite=True)

Wrote 603 bytes.


True

###  Task 1: Ingest and Save

Load the CSV into a PySpark DataFrame.

In [0]:

df = spark.read.option("header", True).option("inferSchema", True) \
.csv("/FileStore/tables/sales_transactions.csv")
df.show()




+--------------+-------------+------+---------------+-------------+--------+----------+----------+
|transaction_id|customer_name|region|        product|     category|quantity|unit_price|      date|
+--------------+-------------+------+---------------+-------------+--------+----------+----------+
|             1|       Rajesh| North|         Laptop|  Electronics|       1|     55000|2024-01-12|
|             2|        Sneha|  West|   Refrigerator|  Electronics|       1|     32000|2024-02-05|
|             3|         Anil| South|        Shampoo|Personal Care|       5|       150|2024-01-17|
|             4|        Divya| North|         Mobile|  Electronics|       2|     20000|2024-03-22|
|             5|       Vikram|  East|Washing Machine|  Electronics|       1|     28000|2024-02-28|
|             6|       Preeti|  West|       Sneakers|      Fashion|       2|      4000|2024-01-31|
|             7|         Aman| South|             TV|  Electronics|       1|     45000|2024-02-15|
|         

 Save it in Parquet and Delta formats (in different folders).

In [0]:
# 2. Save in Parquet format
df.write.mode("overwrite").parquet("/FileStore/tables/sales_transactions_parquet")

# 3. Save in Delta format
delta_path = "/FileStore/tables/sales_transactions_delta"
df.write.format("delta").mode("overwrite").save(delta_path)

Create Delta Table called sales_transactions

In [0]:
spark.sql("USE CATALOG hive_metastore")
spark.sql("DROP TABLE IF EXISTS sales_transactions")
spark.sql(f"""CREATE TABLE sales_transactions USING DELTA LOCATION 'dbfs:{delta_path}'""")

DataFrame[]

### Task 2: Data Transformation

In [0]:
from pyspark.sql.functions import col, month, date_format, when

df = spark.read.format("delta").load(delta_path)

# 1. Add total_amount
df = df.withColumn("total_amount", col("quantity") * col("unit_price"))

# 2. Add month
df = df.withColumn("month", month(col("date")))

# 3. Format date
df = df.withColumn("date_formatted", date_format(col("date"), "dd-MMM-yyyy"))

# 4. Add is_high_value
df = df.withColumn("is_high_value", when(col("total_amount") > 30000, True).otherwise(False))

display(df)


transaction_id,customer_name,region,product,category,quantity,unit_price,date,total_amount,month,date_formatted,is_high_value
1,Rajesh,North,Laptop,Electronics,1,55000,2024-01-12,55000,1,12-Jan-2024,true
2,Sneha,West,Refrigerator,Electronics,1,32000,2024-02-05,32000,2,05-Feb-2024,true
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17,750,1,17-Jan-2024,false
4,Divya,North,Mobile,Electronics,2,20000,2024-03-22,40000,3,22-Mar-2024,true
5,Vikram,East,Washing Machine,Electronics,1,28000,2024-02-28,28000,2,28-Feb-2024,false
6,Preeti,West,Sneakers,Fashion,2,4000,2024-01-31,8000,1,31-Jan-2024,false
7,Aman,South,TV,Electronics,1,45000,2024-02-15,45000,2,15-Feb-2024,true
8,Isha,North,Notebook,Stationery,10,60,2024-01-10,600,1,10-Jan-2024,false
9,Kunal,East,Pencil,Stationery,20,10,2024-03-05,200,3,05-Mar-2024,false
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19,600,3,19-Mar-2024,false


### Tsk 3: Aggregations & Insights

In [0]:
from pyspark.sql.functions import col, month
df = spark.read.format("delta").load(delta_path) \
        .withColumn("total_amount", col("quantity") * col("unit_price")) \
        .withColumn("month", month(col("date")))

# 1. Transactions per region
df.groupBy("region").count().show()

# 2. Top 3 categories by total sales amount
df.groupBy("category").sum("total_amount") \
  .withColumnRenamed("sum(total_amount)", "total_sales") \
  .orderBy(col("total_sales").desc()) \
  .show(3)

# 3. Month-wise revenue trend
df.groupBy("month").sum("total_amount") \
  .withColumnRenamed("sum(total_amount)", "total_sales") \
  .orderBy("month") \
  .show()

# 4. Customer(s) with highest purchase in one transaction
max_amount = df.agg({"total_amount": "max"}).collect()[0][0]
df.filter(col("total_amount") == max_amount) \
  .select("customer_name", "total_amount") \
  .show()

# 5. Total sales done in Q1 (Jan–Mar)
df.filter(col("month").between(1, 3)) \
  .agg({"total_amount": "sum"}) \
  .withColumnRenamed("sum(total_amount)", "Q1_total_sales") \
  .show()


+------+-----+
|region|count|
+------+-----+
|  West|    3|
|  East|    2|
| North|    3|
| South|    2|
+------+-----+

+-------------+-----------+
|     category|total_sales|
+-------------+-----------+
|  Electronics|     200000|
|      Fashion|       8000|
|Personal Care|       1350|
+-------------+-----------+
only showing top 3 rows
+-----+-----------+
|month|total_sales|
+-----+-----------+
|    1|      64350|
|    2|     105000|
|    3|      40800|
+-----+-----------+

+-------------+------------+
|customer_name|total_amount|
+-------------+------------+
|       Rajesh|       55000|
+-------------+------------+

+--------------+
|Q1_total_sales|
+--------------+
|        210150|
+--------------+



###  Task 4: Update & Delete Scenarios

In [0]:
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.functions import col

deltaTable = DeltaTable.forPath(spark, delta_path)

# 1. Update: increase price of Stationery items by 10%
deltaTable.update(
    condition=col("category") == "Stationery",
    set={"unit_price": col("unit_price") * 1.10}
)

# 2. Delete: remove records with quantity < 3
deltaTable.delete(condition=col("quantity") < 3)

# 3. Insert: add today's transaction
new_data = [
    (11, "Satej", "South", "Smartwatch", "Electronics", 1, 15000, datetime.today().strftime("%Y-%m-%d"))
]
update_df = spark.createDataFrame(new_data, [
    "transaction_id", "customer_name", "region", "product", "category", "quantity", "unit_price", "date"
])
deltaTable.alias("target").merge(
    update_df.alias("source"),
    "target.transaction_id = source.transaction_id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

# Show updated table
display(deltaTable.toDF())


transaction_id,customer_name,region,product,category,quantity,unit_price,date
8,Isha,North,Notebook,Stationery,10,66,2024-01-10
9,Kunal,East,Pencil,Stationery,20,11,2024-03-05
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19
11,Satej,South,Smartwatch,Electronics,1,15000,2025-08-08


### Task 5: Partitioning & Optimization (Bonus)

In [0]:
from pyspark.sql.functions import month

# Paths for partitioned tables
delta_partitioned_region = "/tmp/delta_sales_partitioned_region"
delta_partitioned_month = "/tmp/delta_sales_partitioned_month"

# Load Delta table and add month column
df = spark.read.format("delta").load(delta_path) \
        .withColumn("month", month(col("date")))

# 1. Partitioned by region
df.write.format("delta").mode("overwrite").partitionBy("region").save(delta_partitioned_region)
print("Partitioned by region table created.")
display(spark.read.format("delta").load(delta_partitioned_region))

# 2. Partitioned by month
df.write.format("delta").mode("overwrite").partitionBy("month").save(delta_partitioned_month)
print("Partitioned by month table created.")
display(spark.read.format("delta").load(delta_partitioned_month))

# 3. Z-Ordering (Databricks Premium only) – This will output optimization stats
print("Running Z-Ordering on region-partitioned table...")
spark.sql(f"OPTIMIZE delta.`{delta_partitioned_region}` ZORDER BY (category)")



Partitioned by region table created.


transaction_id,customer_name,region,product,category,quantity,unit_price,date,month
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17,1
11,Satej,South,Smartwatch,Electronics,1,15000,2025-08-08,8
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19,3
8,Isha,North,Notebook,Stationery,10,66,2024-01-10,1
9,Kunal,East,Pencil,Stationery,20,11,2024-03-05,3


Partitioned by month table created.


transaction_id,customer_name,region,product,category,quantity,unit_price,date,month
9,Kunal,East,Pencil,Stationery,20,11,2024-03-05,3
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19,3
8,Isha,North,Notebook,Stationery,10,66,2024-01-10,1
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17,1
11,Satej,South,Smartwatch,Electronics,1,15000,2025-08-08,8


Running Z-Ordering on region-partitioned table...


DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,